In [108]:
import numpy as np
import pandas as pd

# Load and preprocess the data

In [109]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv('dataset.csv')

Check for missing values. Fix or drop if it's in SystemCodeNumber. <br>
(Our data set doesn't have any nulls, so that's just in case)

In [110]:
df.dropna(subset=['SystemCodeNumber'])

df[column].fillna(inplace=True, method='backfill')

for column in ['Capacity', 'Occupancy']:
    avg_value = df[column].mean()
    df[column].fillna(avg_value, inplace=True)

Redefine data from 'SystemCodeNumber' column as binary entry in type-columns -- so that it will be treated equally.

In [111]:
df = pd.get_dummies(df, columns=['SystemCodeNumber'])

Split date into 'month', 'day' columns (ignore year - it's const in this data set); split time into h, m, s columns. 

In [112]:
from datetime import datetime

df['LastUpdated'] = pd.to_datetime(df['LastUpdated'], infer_datetime_format=True)

df['LastUpdated_Month'] = [int(dt.month) for dt in df['LastUpdated']]
df['LastUpdated_Day'] = [int(dt.day) for dt in df['LastUpdated']]

df['LastUpdated_Hour'] = [int(dt.hour) for dt in df['LastUpdated']]
df['LastUpdated_Minute'] = [int(dt.minute) for dt in df['LastUpdated']]

df = df.drop(["LastUpdated"], axis=1)

print(df)

       Capacity  Occupancy  SystemCodeNumber_BHMBCCMKT01  \
0           577         61                             1   
1           577         64                             1   
2           577         80                             1   
3           577        107                             1   
4           577        150                             1   
...         ...        ...                           ...   
35712      1920       1517                             0   
35713      1920       1487                             0   
35714      1920       1432                             0   
35715      1920       1321                             0   
35716      1920       1180                             0   

       SystemCodeNumber_BHMBCCPST01  SystemCodeNumber_BHMBCCSNH01  \
0                                 0                             0   
1                                 0                             0   
2                                 0                             0   
3  

In [113]:
# label_enc = LabelEncoder()
# df["SystemCodeNumber"] = label_enc.fit_transform(df["SystemCodeNumber"].values)

# y = df["SystemCodeNumber"]
# onehotencoder = OneHotEncoder(categories='SystemCodeNumber')
# print(onehotencoder.fit_transform(y.reshape(-1,1)))

In [ ]:
# https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

In [114]:
from sklearn.model_selection import train_test_split